In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/CV

/content/drive/MyDrive/Colab Notebooks/CV


In [3]:
# write a efficientNet model to classify adoption rate
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras 
from keras import Sequential, Input
from keras.layers import Conv2D, Dense, UpSampling2D, MaxPool2D, Conv2DTranspose
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model 
import glob
import cv2
from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow as tf
import os

In [4]:
INPUT_SHAPE = (256,256,3)
INPUT_SIZE = INPUT_SHAPE[:2]
BATCH_SIZE = 64

In [5]:
def build_autoencoder_model():
    model = Sequential()
    model.add(Input(shape=INPUT_SHAPE))
    
    # encoder
    model.add(Conv2D(16, 3, (1,1), activation="relu", name="conv1-3x3x16", padding="same"))
    model.add(MaxPool2D((2,2), padding="same"))
    model.add(Conv2D(32, 3, (1,1), activation="relu", name="conv2-3x3x32", padding="same"))
    model.add(MaxPool2D((2,2), padding="same"))
    model.add(Conv2D(64, 3, (1,1), activation="relu", name="conv3-3x3x64", padding="same"))
    model.add(MaxPool2D((2,2), padding="same"))
    model.add(Conv2D(32, 3, (1,1), activation="relu", name="conv4-3x3x32", padding="same"))
    model.add(MaxPool2D((2,2), padding="same"))
    model.add(Conv2D(8, 3, (1,1), activation="relu", name="conv5-3x3x8", padding="same"))
    model.add(MaxPool2D((2,2), padding="same"))
    
    # decoder
    model.add(Conv2D(8, 3, (1,1), activation="relu", name="conv6-3x3x8", padding="same"))
    model.add(UpSampling2D())
    model.add(Conv2D(32, 3, (1,1), activation="relu", name="conv7-3x3x32", padding="same"))
    model.add(UpSampling2D())
    model.add(Conv2D(64, 3, (1,1), activation="relu", name="conv8-3x3x64", padding="same"))
    model.add(UpSampling2D())
    model.add(Conv2D(32, 3, (1,1), activation="relu", name="conv9-3x3x32", padding="same"))
    model.add(UpSampling2D())
    model.add(Conv2D(16, 3, (1,1), activation="relu", name="conv10-3x3x16", padding="same"))
    model.add(UpSampling2D())
    model.add(Conv2D(3, 3, (1,1), activation="relu", name="conv11-3x3x1", padding="same"))
    return model

In [6]:
model = build_autoencoder_model()
model.compile(optimizer="adam", loss=keras.losses.MeanSquaredError())
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1-3x3x16 (Conv2D)        (None, 256, 256, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 16)      0         
_________________________________________________________________
conv2-3x3x32 (Conv2D)        (None, 128, 128, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv3-3x3x64 (Conv2D)        (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv4-3x3x32 (Conv2D)        (None, 32, 32, 32)        1

In [7]:
# load data
datagen = ImageDataGenerator(rescale=1.0/255.0)
train_it = datagen.flow_from_directory("dataset/train/", class_mode="input", batch_size=BATCH_SIZE, target_size=(256,256))
test_it = datagen.flow_from_directory("dataset/test/", class_mode="input", batch_size=BATCH_SIZE, target_size=(256,256))

Found 40510 images belonging to 5 classes.
Found 17494 images belonging to 5 classes.


In [ ]:
history = model.fit(train_it, epochs=100, verbose=1, validation_data=test_it)
model.save("checkpoints/autoencoder/model.100.epochs")

In [ ]:
print(history.history.keys())
# summarize history for loss
plt.plot(np.arange(1,100), history.history['loss'])
plt.plot(np.arange(1,100), history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig("checkpoints/autoencoder/loss_1-100 epochs.png")
plt.show()

In [ ]:
# reload and re-train autoencoder model
# model = load_model("checkpoints/autoencoder/model.10.epochs")
# model.fit(train_it, epochs=10, verbose=1, validation_data=test_it)
# model.save("checkpoints/autoencoder/model.20.epochs")